In [9]:
import os
import joblib
import librosa
import numpy as np
from scipy.io import wavfile
import csv


os.chdir(os.path.dirname(os.path.realpath("__file__")))


print("Current Working Directory:", os.getcwd())

# Load GMM models
gmm_models = []

for i in range(1,31):
    model_path = f"saved_models/gmm_model_{i}.pkl"
    print(model_path)  # Print for debugging purposes

    if os.path.exists(model_path):
        gmm_model = joblib.load(model_path)
        gmm_models.append(gmm_model)
    else:
        print(f"Model file not found: {model_path}")

Current Working Directory: C:\Users\ghosh\Downloads\2020276_kaggle_assignment2
saved_models/gmm_model_1.pkl
saved_models/gmm_model_2.pkl
saved_models/gmm_model_3.pkl
saved_models/gmm_model_4.pkl
saved_models/gmm_model_5.pkl
saved_models/gmm_model_6.pkl
saved_models/gmm_model_7.pkl
saved_models/gmm_model_8.pkl
saved_models/gmm_model_9.pkl
saved_models/gmm_model_10.pkl
saved_models/gmm_model_11.pkl
saved_models/gmm_model_12.pkl
saved_models/gmm_model_13.pkl
saved_models/gmm_model_14.pkl
saved_models/gmm_model_15.pkl
saved_models/gmm_model_16.pkl
saved_models/gmm_model_17.pkl
saved_models/gmm_model_18.pkl
saved_models/gmm_model_19.pkl
saved_models/gmm_model_20.pkl
saved_models/gmm_model_21.pkl
saved_models/gmm_model_22.pkl
saved_models/gmm_model_23.pkl
saved_models/gmm_model_24.pkl
saved_models/gmm_model_25.pkl
saved_models/gmm_model_26.pkl
saved_models/gmm_model_27.pkl
saved_models/gmm_model_28.pkl
saved_models/gmm_model_29.pkl
saved_models/gmm_model_30.pkl


C:\Users\ghosh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator GaussianMixture from version 1.3.0 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [3]:

user_input = input("T for text file, P for path, C for CSV file: ")

In [6]:
class_mapping = {'right': 0, 'eight': 1, 'cat': 2, 'tree': 3, 'bed': 4, 'happy': 5, 'go': 6, 'dog': 7, 'no': 8, 'wow': 9,
                 'nine': 10, 'left': 11, 'stop': 12, 'three': 13, 'sheila': 14, 'one': 15, 'bird': 16, 'zero': 17,
                 'seven': 18, 'up': 19, 'marvin': 20, 'two': 21, 'house': 22, 'down': 23, 'six': 24, 'yes': 25,
                 'on': 26, 'five': 27, 'off': 28, 'four': 29}
fclass_mapping = {v: k for k, v in class_mapping.items()}

In [8]:
inp=input("Y if file from speech command test else N: ")
if user_input == "T" or user_input == "t":
    file_path = input("Enter the path to the text file: ").strip('"')
    try:
        with open(file_path, "r") as f:
            file_paths = f.readlines()
            file_paths = [x.strip() for x in file_paths]
        print("File paths read successfully.")
        if(inp=="Y" or inp=="y"):
            file_paths = [f"C:\\Users\\ghosh\\Downloads\\SpeechCommandTest-20231114T162637Z-001\\SpeechCommandTest\\{x}" for x in file_paths]
        ans_prob=[]
        for j in file_paths:
            temp_prob=[]
            for i in gmm_models:
                sample_rate, audio = wavfile.read(j)
                audio = audio.astype(np.float32)
                mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=13)
                delta_mfccs = librosa.feature.delta(mfccs)

                delta2_mfccs = librosa.feature.delta(mfccs, order=2)

                combined_features = np.concatenate((mfccs, delta_mfccs))
                combined_features = np.concatenate((combined_features, delta2_mfccs))
                features=combined_features.transpose()
                temp_prob.append(i.score(features))
            ans_prob.append(temp_prob.index(max(temp_prob)))
        for i in ans_prob:
            print(str(i-1)+" "+fclass_mapping[i-1])          
    except OSError as e:
        print(f"Error reading file: {e}")
elif(user_input == "P" or user_input == "p"):
    file_path = input("Enter the path to the folder: ")
    temp_prob=[]
    for i in gmm_models:
        sample_rate, audio = wavfile.read(file_path)
        audio = audio.astype(np.float32)
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=13)
        delta_mfccs = librosa.feature.delta(mfccs)

        delta2_mfccs = librosa.feature.delta(mfccs, order=2)

        combined_features = np.concatenate((mfccs, delta_mfccs))
        combined_features = np.concatenate((combined_features, delta2_mfccs))
        features=combined_features.transpose()
        temp_prob.append(i.score(features))
            
    print(str(temp_prob.index(max(temp_prob)-1))+" "+fclass_mapping[temp_prob.index(max(temp_prob)-1)])
elif(user_input == "C" or user_input == "c"):
    file_path = input("Enter the path to the CSV file: ").strip('"')
    print(file_path)
    with open(file_path, 'r') as csv_file:
            csv_reader = csv.DictReader(csv_file)

            # Assuming 'AUDIO_FILE' column exists in the CSV
            file_paths = [row['AUDIO_FILE'] for row in csv_reader]
            if(inp=="Y" or inp=="y"):
                file_paths = [f"C:\\Users\\ghosh\\Downloads\\SpeechCommandTest-20231114T162637Z-001\\SpeechCommandTest\\{x}" for x in file_paths]
            ans_prob = []

            for j in file_paths:
                temp_prob = []

                for i in gmm_models:
                    sample_rate, audio = wavfile.read(j)
                    audio = audio.astype(np.float32)
                    mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=13)
                    delta_mfccs = librosa.feature.delta(mfccs)

                    delta2_mfccs = librosa.feature.delta(mfccs, order=2)

                    combined_features = np.concatenate((mfccs, delta_mfccs))
                    combined_features = np.concatenate((combined_features, delta2_mfccs))
                    features = combined_features.transpose()
                    temp_prob.append(i.score(features))

                ans_prob.append(temp_prob.index(max(temp_prob)))

            for i in ans_prob:
                print(str(i-1) + " " + fclass_mapping[i-1])

File paths read successfully.
5 happy
5 happy
24 six
